In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_adapter import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

/root/MetaLifelongLanguage/env-adapter/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904898.pt"
# model_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774382.pt"
# model_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782801.pt"
# model_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497477.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-02-13-srev-curri/OML-order1-id4-2023-02-13_05-46-13.147873.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-23-lorav2/OML-order1-id4-2023-02-23_23-07-12.342518.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr001-2023-02-26-lorav6/OML-order1-id4-2023-02-27_01-31-30.024416.pt"
model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2023-03-04-lorav8/OML-order1-id4-2023-03-04_07-28-45.624912.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904925_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774413_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782827_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303088_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497510_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2023-03-04-lorav8/OML-order1-id4-2023-03-04_07-28-45.624964_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 1e-3*2,
    "inner_adapter_lr": 1e-4*20,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "1fc",
    "adapter": True
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
lora                     bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               1


2023-03-04 07:50:05,370 - OML-Log - INFO - Loaded TransformerRLN as RLN
2023-03-04 07:50:05,375 - OML-Log - INFO - Loaded LinearPLN as PLN
/root/MetaLifelongLanguage/env-adapter/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.train()
    learner.pln.train()
    # Enable adapter training (Freeze all Others that isn't LoRA)
    learner.rln.encoder.train_adapter("lora")

    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.combinedRLNPLN, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (frlnpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                output = frlnpln(input_dict)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_output = frlnpln(query_input_dict) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.7227, accuracy = 0.7375, precision = 0.7689, recall = 0.7267, F1 score = 0.7186
Query set metrics: Loss = 0.5154, accuracy = 0.7500, precision = 0.7767, recall = 0.7267, F1 score = 0.7314
Query ID 1/475
Support set metrics: Loss = 0.5976, accuracy = 0.7625, precision = 0.7870, recall = 0.7500, F1 score = 0.7546
Query set metrics: Loss = 0.9317, accuracy = 0.7500, precision = 0.7333, recall = 0.8600, F1 score = 0.7311
Query ID 2/475
Support set metrics: Loss = 0.5038, accuracy = 0.7875, precision = 0.8194, recall = 0.7733, F1 score = 0.7742
Query set metrics: Loss = 0.9693, accuracy = 0.7500, precision = 0.8476, recall = 0.7833, F1 score = 0.7455
Query ID 3/475
Support set metrics: Loss = 0.5705, accuracy = 0.7250, precision = 0.7569, recall = 0.7100, F1 score = 0.6981
Query set metrics: Loss = 1.0290, accuracy = 0.6250, precision = 0.6167, recall = 0.6500, F1 score = 

Support set metrics: Loss = 0.6630, accuracy = 0.7000, precision = 0.7524, recall = 0.6900, F1 score = 0.6921
Query set metrics: Loss = 0.7674, accuracy = 0.6250, precision = 0.7054, recall = 0.6375, F1 score = 0.6458
Query ID 36/475
Support set metrics: Loss = 0.5750, accuracy = 0.7500, precision = 0.7950, recall = 0.7433, F1 score = 0.7470
Query set metrics: Loss = 1.1983, accuracy = 0.6250, precision = 0.3714, recall = 0.5500, F1 score = 0.4146
Query ID 37/475
Support set metrics: Loss = 0.6298, accuracy = 0.7125, precision = 0.7448, recall = 0.7000, F1 score = 0.6975
Query set metrics: Loss = 0.7932, accuracy = 0.5625, precision = 0.4429, recall = 0.5667, F1 score = 0.4800
Query ID 38/475
Support set metrics: Loss = 0.5772, accuracy = 0.7875, precision = 0.8234, recall = 0.7767, F1 score = 0.7812
Query set metrics: Loss = 1.3755, accuracy = 0.5000, precision = 0.3667, recall = 0.4800, F1 score = 0.3933
Query ID 39/475
Support set metrics: Loss = 0.4769, accuracy = 0.7875, precision

Query set metrics: Loss = 1.3854, accuracy = 0.4375, precision = 0.4667, recall = 0.4267, F1 score = 0.3949
Query ID 71/475
Support set metrics: Loss = 0.6389, accuracy = 0.7500, precision = 0.7771, recall = 0.7333, F1 score = 0.7267
Query set metrics: Loss = 1.0124, accuracy = 0.6875, precision = 0.6333, recall = 0.6000, F1 score = 0.5614
Query ID 72/475
Support set metrics: Loss = 0.6176, accuracy = 0.7500, precision = 0.7778, recall = 0.7333, F1 score = 0.7060
Query set metrics: Loss = 1.1625, accuracy = 0.6250, precision = 0.7250, recall = 0.7300, F1 score = 0.6338
Query ID 73/475
Support set metrics: Loss = 0.6411, accuracy = 0.7625, precision = 0.7917, recall = 0.7533, F1 score = 0.7555
Query set metrics: Loss = 1.1295, accuracy = 0.6250, precision = 0.7067, recall = 0.7033, F1 score = 0.6100
Query ID 74/475
Support set metrics: Loss = 0.6463, accuracy = 0.7625, precision = 0.8220, recall = 0.7533, F1 score = 0.7581
Query set metrics: Loss = 1.3951, accuracy = 0.5625, precision =

Support set metrics: Loss = 0.7083, accuracy = 0.7750, precision = 0.8219, recall = 0.7633, F1 score = 0.7765
Query set metrics: Loss = 1.1665, accuracy = 0.4375, precision = 0.4800, recall = 0.5667, F1 score = 0.4776
Query ID 107/475
Support set metrics: Loss = 0.8321, accuracy = 0.6875, precision = 0.7591, recall = 0.6667, F1 score = 0.6818
Query set metrics: Loss = 1.1115, accuracy = 0.4375, precision = 0.3400, recall = 0.4000, F1 score = 0.3500
Query ID 108/475
Support set metrics: Loss = 0.6067, accuracy = 0.7375, precision = 0.7865, recall = 0.7200, F1 score = 0.6951
Query set metrics: Loss = 1.0644, accuracy = 0.4375, precision = 0.3667, recall = 0.4133, F1 score = 0.3365
Query ID 109/475
Support set metrics: Loss = 0.6063, accuracy = 0.7250, precision = 0.7369, recall = 0.7067, F1 score = 0.6903
Query set metrics: Loss = 1.0559, accuracy = 0.5625, precision = 0.6133, recall = 0.6633, F1 score = 0.5619
Query ID 110/475
Support set metrics: Loss = 0.6410, accuracy = 0.7125, preci

Query ID 141/475
Support set metrics: Loss = 0.7118, accuracy = 0.7000, precision = 0.7214, recall = 0.6867, F1 score = 0.6801
Query set metrics: Loss = 0.6938, accuracy = 0.6250, precision = 0.6600, recall = 0.6000, F1 score = 0.5744
Query ID 142/475
Support set metrics: Loss = 0.6576, accuracy = 0.7125, precision = 0.7675, recall = 0.7033, F1 score = 0.7027
Query set metrics: Loss = 0.7681, accuracy = 0.6250, precision = 0.5143, recall = 0.6200, F1 score = 0.5097
Query ID 143/475
Support set metrics: Loss = 0.5891, accuracy = 0.7625, precision = 0.7757, recall = 0.7500, F1 score = 0.7446
Query set metrics: Loss = 0.9216, accuracy = 0.7500, precision = 0.7762, recall = 0.7500, F1 score = 0.7067
Query ID 144/475
Support set metrics: Loss = 0.5219, accuracy = 0.7625, precision = 0.8011, recall = 0.7533, F1 score = 0.7554
Query set metrics: Loss = 1.3808, accuracy = 0.3750, precision = 0.4267, recall = 0.3500, F1 score = 0.3143
Query ID 145/475
Support set metrics: Loss = 0.6657, accurac

...

Support set metrics: Loss = 0.4624, accuracy = 0.9250, precision = 0.9332, recall = 0.9200, F1 score = 0.9161
Query set metrics: Loss = 0.5204, accuracy = 0.8750, precision = 0.9062, recall = 0.9062, F1 score = 0.8810
Query ID 313/475
Support set metrics: Loss = 0.4153, accuracy = 0.9250, precision = 0.9448, recall = 0.9300, F1 score = 0.9349
Query set metrics: Loss = 0.4372, accuracy = 0.8750, precision = 0.8958, recall = 0.9333, F1 score = 0.9028
Query ID 314/475
Support set metrics: Loss = 0.4789, accuracy = 0.9250, precision = 0.9394, recall = 0.9000, F1 score = 0.9025
Query set metrics: Loss = 0.4152, accuracy = 0.8750, precision = 0.8095, recall = 0.8095, F1 score = 0.8095
Query ID 315/475
Support set metrics: Loss = 0.4761, accuracy = 0.8625, precision = 0.8687, recall = 0.8300, F1 score = 0.8310
Query set metrics: Loss = 0.7930, accuracy = 0.8125, precision = 0.9286, recall = 0.8333, F1 score = 0.8442
Query ID 316/475
Support set metrics: Loss = 0.4382, accuracy = 0.8625, preci

Support set metrics: Loss = 0.4397, accuracy = 0.9375, precision = 0.9452, recall = 0.9200, F1 score = 0.9253
Query set metrics: Loss = 0.6247, accuracy = 0.8125, precision = 0.7292, recall = 0.7500, F1 score = 0.7083
Query ID 348/475
Support set metrics: Loss = 0.4296, accuracy = 0.9250, precision = 0.9199, recall = 0.9200, F1 score = 0.9077
Query set metrics: Loss = 0.8779, accuracy = 0.7500, precision = 0.6481, recall = 0.6667, F1 score = 0.6296
Query ID 349/475
Support set metrics: Loss = 0.4689, accuracy = 0.8750, precision = 0.8766, recall = 0.8700, F1 score = 0.8691
Query set metrics: Loss = 0.7486, accuracy = 0.8125, precision = 0.7857, recall = 0.7857, F1 score = 0.7619
Query ID 350/475
Support set metrics: Loss = 0.4795, accuracy = 0.9250, precision = 0.9381, recall = 0.9100, F1 score = 0.9127
Query set metrics: Loss = 0.5138, accuracy = 0.8125, precision = 0.8333, recall = 0.7708, F1 score = 0.7917
Query ID 351/475
Support set metrics: Loss = 0.4677, accuracy = 0.9250, preci

Support set metrics: Loss = 0.4390, accuracy = 0.9375, precision = 0.9257, recall = 0.9300, F1 score = 0.9220
Query set metrics: Loss = 0.7918, accuracy = 0.8125, precision = 0.7619, recall = 0.7143, F1 score = 0.6905
Query ID 383/475
Support set metrics: Loss = 0.4132, accuracy = 0.9500, precision = 0.9618, recall = 0.9400, F1 score = 0.9455
Query set metrics: Loss = 0.6622, accuracy = 0.8750, precision = 0.9286, recall = 0.9238, F1 score = 0.9079
Query ID 384/475
Support set metrics: Loss = 0.3953, accuracy = 0.9250, precision = 0.9436, recall = 0.9000, F1 score = 0.8916
Query set metrics: Loss = 1.2105, accuracy = 0.6250, precision = 0.5833, recall = 0.6500, F1 score = 0.5633
Query ID 385/475
Support set metrics: Loss = 0.4196, accuracy = 0.9500, precision = 0.9561, recall = 0.9300, F1 score = 0.9380
Query set metrics: Loss = 0.9127, accuracy = 0.6875, precision = 0.8125, recall = 0.6979, F1 score = 0.7238
Query ID 386/475
Support set metrics: Loss = 0.4712, accuracy = 0.8875, preci

Support set metrics: Loss = 0.4900, accuracy = 0.9125, precision = 0.9247, recall = 0.9100, F1 score = 0.9093
Query set metrics: Loss = 0.6544, accuracy = 0.8125, precision = 0.8000, recall = 0.8250, F1 score = 0.7857
Query ID 418/475
Support set metrics: Loss = 0.4376, accuracy = 0.9500, precision = 0.9489, recall = 0.9600, F1 score = 0.9511
Query set metrics: Loss = 0.7984, accuracy = 0.7500, precision = 0.8542, recall = 0.8229, F1 score = 0.7946
Query ID 419/475
Support set metrics: Loss = 0.4281, accuracy = 0.9500, precision = 0.9627, recall = 0.9300, F1 score = 0.9396
Query set metrics: Loss = 0.9485, accuracy = 0.5625, precision = 0.7500, recall = 0.5000, F1 score = 0.5833
Query ID 420/475
Support set metrics: Loss = 0.4616, accuracy = 0.9125, precision = 0.9031, recall = 0.8900, F1 score = 0.8902
Query set metrics: Loss = 1.0041, accuracy = 0.6875, precision = 0.4815, recall = 0.6111, F1 score = 0.5111
Query ID 421/475
Support set metrics: Loss = 0.4637, accuracy = 0.9125, preci

Support set metrics: Loss = 0.3904, accuracy = 0.9500, precision = 0.9525, recall = 0.9500, F1 score = 0.9453
Query set metrics: Loss = 0.1878, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 453/475
Support set metrics: Loss = 0.4092, accuracy = 0.9500, precision = 0.9552, recall = 0.9500, F1 score = 0.9497
Query set metrics: Loss = 0.5124, accuracy = 0.8125, precision = 0.8750, recall = 0.8646, F1 score = 0.8530
Query ID 454/475
Support set metrics: Loss = 0.4508, accuracy = 0.8875, precision = 0.8844, recall = 0.8600, F1 score = 0.8627
Query set metrics: Loss = 0.5072, accuracy = 0.8125, precision = 0.7500, recall = 0.8000, F1 score = 0.7667
Query ID 455/475
Support set metrics: Loss = 0.4095, accuracy = 0.9625, precision = 0.9652, recall = 0.9600, F1 score = 0.9597
Query set metrics: Loss = 0.6093, accuracy = 0.6875, precision = 0.5481, recall = 0.5926, F1 score = 0.5574
Query ID 456/475
Support set metrics: Loss = 0.5389, accuracy = 0.8875, preci

In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-inlr002-2023-03-04-lorav8/OML-order1-id4-2023-03-04_07-28-45.624912_update5_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order1-inlr002-2023-03-04-lorav8/OML-order1-id4-2023-03-04_07-28-45.624912_update5_time_inference.csv
